#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 05 - Containers 1 - Exercises</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### 01 Exercises

This exercise will cover creating a simple Docker application for doing some simple file operations. We'll cover building, running, interacting with the host file system, managing running containers, and other admins operations.

### Installation

The first thing is install Docker if you don't already have it. 

* **Linux**: Most of our workstations are Ubuntu so follow the instructions [here](https://docs.docker.com/engine/install/ubuntu). Other Linuces (Linuxes?) are [supported](https://docs.docker.com/engine/install/#server) but might be harder to trouble shoot.


* **Windows**: Instructions are [here](https://docs.docker.com/docker-for-windows/install). This should be a simple application installation, just be sure to [switch to Linux containers](https://docs.docker.com/docker-for-windows/#switch-between-windows-and-linux-containers).


* **macOS**: Instructions are [here](https://docs.docker.com/docker-for-mac/install). This should also be straight-forward, however I (Eric) don't have a Mac with me so it might be difficult trouble shooting if anything comes up, using another machine or logging remotely into a Ubuntu workstation on the KCL network may be better.

### Exercise 1: 

Let's create a Docker app which inverts the colours of images in a folder, placing the results in "inverted" subfolder. We'll use [ImageMagick](https://imagemagick.org) and a simple bash script to do this. 

**Step 1:** Create a directory called `exercise01` and copy the following into it as `invert.sh`:

```sh
#! /bin/bash

# files are read from this directory
INPUT_DIR=/input

# create the output directory
mkdir -p $INPUT_DIR/inverted

# iterate over every file with an extension in INPUT_DIR and convert it, this will fail for non-images of course
for i in $INPUT_DIR/*.*
do
    # if $i is a file attempt to convert it
    if [ -f "$i" ]
    then
        outfile=$INPUT_DIR/inverted/$(basename "$i")
        convert -channel RGB -negate "$i" "$outfile"
    fi
done
```

**Step 2:** We now want to define a `Dockerfile` in the `exercise01` directory. This file will need to do the following:

* Specify the base image (I'd suggest some flavour of Ubuntu)

* Install ImageMagick (use `apt` for this, although you could download the pre-built `magick` executable from their site and use `magick convert`)

* Copy the script to the root directory

* Set the command to run the script when the container is run

Create and fill in your `Dockerfile` to accomplish this, the reference for commands is [here](https://docs.docker.com/engine/reference/builder).

**Step 3:** Build your image using the `docker build` command, choosing to tag it as `image-invert`. If you need help on the command `docker build --help` will list options and usage information.

One reason why we've created a directory to do the building in is that Docker will copy everything in a build directory to a temporary location, so having lots of large files where your Dockerfile lives isn't a good idea.

**Step 4:** Now we want to run our app and convert some images. In the workshop directory we have the directory `testimages`, but how do we give the container access to this directory to read from and write to? The script mentions a directory called `/input` for reading and writing, but that doesn't exist in the image.

One way external data is accessed in Docker is through binding volumes, which in our case making a local directory accessible under a given name. The documentation on this is [here](https://docs.docker.com/storage/bind-mounts/#start-a-container-with-a-bind-mount). 

Run your Docker image from the workshop directory with the added argument `-v /full/path/to/testimages:/input`. This creates a volume binding associating `/full/path/to/testimages` (ie. the absolute path to `testimages`) to `/input` in the container. After running your container check `testimages` to see what's there now.

The alternative and more current syntax for command line arguments to do binding is with `--bind`, for us this would look like `--mount src=/full/path/to/testimages,target=/input,type=bind`.

**Step 5:** If you're running on Linux you'll notice that the created directory in `testimages` is owned by root. This is because the Docker daemon runs as root and the command within the container also runs as root. It's not good or secure practice to allow a container command to run internally as root so we have to create an internal user account and run as that.

Delete the output directory using `sudo rm -rf testimages/inverted`.

In your `Dockerfile` before the `CMD` line at the end, add the following:

```dockerfile
RUN adduser dockeruser --shell /bin/bash
USER dockeruser
```

What these commands do is run `adduser` to create an internal user account, then `USER` states the following commands will be run as that user in the container. Now the script file will be run as `dockeruser`, whose user ID (UID) and group ID (GID) don't match anything on the host system. 

Using a non-root user internally, one without elevated privileges, has solved our security concern but not the file ownership one. Attempting to run the container now with the same command will result in error if `testimages` doesn't have permissions set to allow any user to write to it (which is the default).

The solution now is to use the `--user`/`-u` flag to specify the user under which to run the container. The syntax we'll use with this flag is `UID:GID` which you can get with the `id` command, so add `-u $(id -u):$(id -g)` to your `docker run` command and run the container again. Your generated images should be owned by you now.

### Exercise 2:

The docker command provides mechanisms for managing images and containers. By now you'll have at least one container that's been run, if you used `--rm` with your run commands they'll be stopped once the command completes but will persist. If something is still running in the background (perhaps your `hello-flask` server you made from the notes then forgot about) you can see this with `docker ps`:

```sh
$ docker ps
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
1cdc1e2ee834        hello-flask         "flask run --host=0.…"   29 hours ago        Up 29 hours                             quizzical_lichterman

```

The docker command line documentation is [here](https://docs.docker.com/engine/reference/commandline/cli) specifically covering this command and others. In this exercise  find the commands to kill any running containers you don't want anymore and clean up any finished containers kicking around. Images that aren't being used can also be cleaned up if you have any. 